**Ch05-13: Pandas入门及数据预处理！！**

Source: 
[数据分析实战 45 讲](https://time.geekbang.org/column/intro/100021701?tab=intro)

---

## Series与DataFrame

Series是个定长的字典序列，存储的时候相当于两个 ndarray，这也是和字典（元素个数不固定）最大的不同。Series有两个基本属性：index和values。可以`Series(data=[],index=[])`创建（可不指定index，即平平无奇数字索引），也可以采用字典的方式`Series({})`创建。

可以将DataFrame看成是由相同索引的Series**按列叠起来**组成的字典类型，所以**此时字典的键是对应列索引**的，而**行索引由Series自己的index决定**（见下面）。

- 也请注意“由相同索引的Series”这个限定条件，因为如果索引不同的话会像底下df2那样子。

pandas允许直接从xlsx和csv等文件中读入数据（eg. `pd.read_csv()`，也可以写入到这些文件中。

### DataFrame访问

- `DataFrame.loc['row','col']`/`DataFrame.iloc[row_index,col_index]`属于**单步赋值**；`df['col']['row']`属于**链式赋值**，赋值后会自动生成副本，不是修改原数据。

    - 一定一定注意！！！对于链式赋值是先列后行！！！！！相反的！！！！

- 链式赋值时不可以直接用`[:]`进行切片，想用冒号必须得单步赋值，eg. `df.iloc[1:3]`或`df.loc[:, 'B':'D']`。

- 遍历df的时候用列索引组成的list最方便。

## 函数/方法

- 使用`Series.method()`、`DataFrame.method()`以及`pd.function()`时（除非`inplace=True`）会产生新副本，所以想修改原数据的话要记得assign back。

- Pandas和NumPy一样，都有常用的统计函数，并且如果遇到空值NaN会自动排除。`DataFrame.describe()`可以一次性输出很多统计值！

- 区分一下这三种函数（底下还有例子）。它们的效果给人一种numpy ufunc的感觉（吗），不用for循环逐元素遍历。

    - `.apply(func)`: **more versatile; flexible output shape**. Allows for axis control (row-wise by `axis=1` or column-wise by `axis=0`) when applied on a DataFrame.

    - `.map(func)`: simple **element-wise** operations. `DataFrame.map` is renamed from `DataFrame.applymap`; when using this, the function applied *must return a single value from a single value* (so aggregate functions not permitted).

    - `.transform(func)`: **often used with `groupby()`** for *aggregation functions*, but strictly ensures the output shape matches the input.
 
    - `func`不用后面的括号。

### DataFrame.method()中奇怪的axis

pandas里面指定某方向真的好奇怪，有些跟numpy是相反的有些好像又不是，，，

* “`axis=0` (default): refers to operations **along rows**, meaning it applies column-wise (in the **vertical** direction). Note that for *operation methods* like `DataFrame.drop()` and `DataFrame.rename()` they're exerted **on rows**, due to the aforementioned *"along rows"*. 

### pandas与SQL

- 数据表合并：`pd.merge()`函数的`how=`参数分'inner'/'left'/'right'/'outer'，拼接方式等同于SQL中的INNER/LEFT/RIGHT/OUTER JOIN，而且也是用`on=`参数指定匹配依据的列。

- pandasql包：在python中直接使用SQL queries操作DataFrame。pandasql中的主要函数是`sqldf()`，它接收两个参数：一个SQL语句，还有一组环境变量`globals()`或`locals()`。使用匿名函数：
  
```Python
pysqldf = lambda sql: sqldf(sql, globals())
sql = "SELECT * FROM df1 WHERE name ='_'"
print(pysqldf(sql))
```
---

# pandas与数据清洗

有时候先用`DataFrame.astype()`把格式转成str类型是为了方便对数据进行操作，因为有好多好用的`Series.str.method()`，比如可以用`strip()`删除数据间的空格或其他指定字符、大小写转换等。

**数据清洗可总结为四个关键点 “完全合一”：**

1. **完**整性：是否存在空值，字段是否完整。`Series.fillna()`对**某字段的缺失值**进行填充，`DataFrame.dropna(how='all')`删除全部**空行**。

```Python
# 用众数填充缺失字段
data = {'A': [1, 2, 2, np.nan, 3],'B': [np.nan, 'x', 'y', 'x', 'x']}
df = pd.DataFrame(data)
for column in df.columns:  # 这样子遍历好用
    mode_value = df[column].mode()[0]  # 获取众数，注意对于数值类型还是非数值类型都能用！！
    df[column] = df[column].fillna(mode_value)  
```

2. **全**面性：观察某列数值，瞪眼法或者通过几个统计值（eg. 均值/最值）判断该列是否有问题，比如**列数据的单位不统一**。

```Python
# 单位不统一时，将磅（lbs）转化为千克（kg）：
rows_with_lbs = df['weight'].str.contains('lbs').fillna(False) # 获取weight列中单位为lbs的数据
print(df[rows_with_lbs])
for i,lbs_row in df[rows_with_lbs].iterrows(): # DataFrame.iterrows还会返回行索引！
  # 截取从头开始到倒数第三个字符之前，即去掉lbs。再将lbs转换为kg, 2.2lbs=1kg
  weight = int(float(lbs_row['weight'][:-3])/2.2)
  df.at[i,'weight'] = '{}kgs'.format(weight) 
```

3. **合**法性：数据的类型、内容、大小是否合法，比如是否存在非ASCII字符、年龄超过150岁等。

```Python
# 删除非 ASCII 字符
df['first_name'] = df['first_name'].replace({r'[^\x00-\x7F]+':''}, regex=True)
df['last_name'] = df['last_name'].replace({r'[^\x00-\x7F]+':''}, regex=True)
```

4. **唯**一性：数据是否存在重复记录；行数据、列数据都需要是唯一的，列的话可以使用`Series.str.split`切分并`DataFrame.drop('col', axis=1)`删除原数据列，行的话就是`DataFrame.drop_duplicates('col')`啦。

```Python
# 切分名字，删除源数据列
df[['first_name','last_name']] = df['name'].str.split(expand=True)
df = df.drop('name', axis=1)
```
---

# 数据规范化

## sklearn.preprocessing

**一条极度personalized的reminder：scaler千万别再拼错成scalar我都服了怎么可以这么笨**

从贝叶斯分类器那里复制过来：

`transform()`和`fit_transform()`二者的功能都是对数据进行某种统一处理（比如标准化，将数据缩放(映射)到某个固定区间等等）。

- `fit_transform(trainData)`对训练数据先**拟合**fit，找到该部分的整体指标，如均值、方差等，然后对该trainData进行**转换**transform，从而实现数据的标准化、归一化等等。

- 根据对之前trainData进行fit后的整体指标！！对剩余的数据（testData）使用同样的均值、方差等指标进行转换`transform(testData)`，**从而保证train、test处理方式相同**！！！！

### 数据规范化的几种方法

最底下是代码示例！

1. **Min-max规范化**：把原始数据映射到指定空间`[min, max]`，默认为`[0,1]`。

2. **Z-score规范化**：没错就是统计里面那个test statistic！！（原数值 - 均值）/ SD，但Z-score的不足就在于它除了用于比较外结果没什么实际意义。 sklearn中`preprocessing.scale()`可以直接给数据这样子规范化，`preprocessing.StandardScalar()`类（注意命名！）功能相同，但这个需要先实例化创建一个对象再用到上述的`fit_transform`。

3. **小数定标规范化**：通过移动小数点的位置来进行规范化，小数点移动多少位取决于该属性取值中的最大绝对值，比如属性A的取值范围是-999到88，那么最大绝对值为999，小数点就会移动3位，即新数值 = 原数值 /1000，即A的取值范围被规范化为-0.999到0.088。

In [20]:
import pandas as pd

# Creating Series
a1 = pd.Series([1,2,3,4])
a2 = pd.Series(data=[1,2,3,4], index=['a','b','c','d'])
a3 = pd.Series({'e':5,'f':6,'g':7,'h':8}) # 也可以采用字典的方式来创建 Series
print("未指定索引名:\n",a1)
print(a2)
print(a3)

# Creating a df by passing a dict (stacked as columns!!!)
df1 = pd.DataFrame({'Col_1':a1, # 毕竟a1是未指定索引名的嘛
                    'Col_2':range(4)})
print(df1)
print(pd.DataFrame({'Col_1':a1,'Col_2':a2}),"\n所以果然Series得索引相同才能叠一起")                  

未指定索引名:
 0    1
1    2
2    3
3    4
dtype: int64
a    1
b    2
c    3
d    4
dtype: int64
e    5
f    6
g    7
h    8
dtype: int64
   Col_1  Col_2
0      1      0
1      2      1
2      3      2
3      4      3
   Col_1  Col_2
0    1.0    NaN
1    2.0    NaN
2    3.0    NaN
3    4.0    NaN
a    NaN    1.0
b    NaN    2.0
c    NaN    3.0
d    NaN    4.0 
所以果然Series得索引相同才能叠一起


In [55]:
# apply(), map(), transform()

df = pd.DataFrame({'A': [1, 1, 2], 'B': [3, 4, 5]})

df_apply = df.apply(sum, axis=0)
df_map_dict = df['A'].map({1:'a',4:'b'}) # mapping using a dict
df_transform = df.groupby('A').transform("sum")
print(df_apply, "Flexible output shape")
print("\n",df_map_dict)
print("\n",df_transform,"Output shape strictly matches input shape, so '7' (grouped by 'A') repeated twice")

A     4
B    12
dtype: int64 Flexible output shape

 0      a
1      a
2    NaN
Name: A, dtype: object

    B
0  7
1  7
2  5 Output shape strictly matches input shape, so '7' (grouped by 'A') repeated twice


In [69]:
# 数据规范化！！！
from sklearn import preprocessing
import numpy as np

x = np.array([[ 0., -3.,  1.],
              [ 3.,  1.,  2.],
              [ 0.,  1., -1.]])

# Min-max规范化，默认映射到[0,1]
min_max_scaler = preprocessing.MinMaxScaler()
minmax_x = min_max_scaler.fit_transform(x)
print(minmax_x)

# Z-Score规范化
scaled_x = preprocessing.scale(x)
print(scaled_x)

# 小数定标规范化
j = np.ceil(np.log10(np.max(abs(x))))
scaled_x2 = x/(10**j)
print(scaled_x2)

[[0.         0.         0.66666667]
 [1.         1.         1.        ]
 [0.         1.         0.        ]]
[[-0.70710678 -1.41421356  0.26726124]
 [ 1.41421356  0.70710678  1.06904497]
 [-0.70710678  0.70710678 -1.33630621]]
[[ 0.  -0.3  0.1]
 [ 0.3  0.1  0.2]
 [ 0.   0.1 -0.1]]
